In [2]:
from datasets import load_from_disk
import datasets

Concatenate datasets

In [3]:
data_folders = ["./annotations/annotations_ru_20"]
data = load_from_disk(data_folders[0])
for folder in data_folders[1:]:
    data = datasets.concatenate_datasets([data, load_from_disk(folder)])


In [4]:
print(f'columns: {data.column_names}')
print(f'nrows: {data.num_rows}')

columns: ['doc_id', 'seg_id', 'translation', 'context']
nrows: 20


In [5]:
print(data[0])

{'doc_id': 1695, 'seg_id': 205, 'translation': {'en': 'What is it about you?', 'nl': 'Wat heb jij?'}, 'context': {'en': ['CA: How cool is that?  Elon, how have you done this?', "These projects are so -- Paypal, SolarCity, Tesla, SpaceX, they're so spectacularly different, they're such ambitious projects at scale.", 'How on Earth has one person been able to innovate in this way?'], 'nl': ['CA: Hoe gaaf is data  Elon, hoe heb je dit gedaan?', 'Deze projecten zijn zo -- PayPal, SolarCity, Tesla, SpaceX, ze zijn zo spectaculair anders, het zijn zeer grote, ambitieuze projecten.', 'Hoe is het in hemelsnaam mogelijk, dat één persoon zó kan innoveren?'], 'ru': ['КА: Насколько это здорово? Элон, как вы это сделали?', 'Эти проекты такие — Paypal, SolarCity, Tesla, SpaceX, они настолько впечатляюще разные, они такие амбициозные проекты по масштабу.', 'Как это возможно чтобы один человек смог внедрить инновации таким образом?']}}


Context is stored as list of sentences, concatenate to a single, long string.

In [6]:
def concatenate_context(row, languages=None):
    if languages is None:
        languages = ['en', 'nl', 'ru']
    row['context_concatenated'] = {
        lan: ' '.join(row['context'][lan]) for lan in languages
    }
    return row

In [7]:
data = data.map(concatenate_context)

In [8]:
for row in data:
    print(row['context_concatenated']['en'])
    print(row['context_concatenated']['nl'])
    print(row['context_concatenated']['ru'])
    print(row['translation']['en'])
    print(row['translation']['nl'])
    # print(row['translation']['ru'])
    print()

CA: How cool is that?  Elon, how have you done this? These projects are so -- Paypal, SolarCity, Tesla, SpaceX, they're so spectacularly different, they're such ambitious projects at scale. How on Earth has one person been able to innovate in this way?
CA: Hoe gaaf is data  Elon, hoe heb je dit gedaan? Deze projecten zijn zo -- PayPal, SolarCity, Tesla, SpaceX, ze zijn zo spectaculair anders, het zijn zeer grote, ambitieuze projecten. Hoe is het in hemelsnaam mogelijk, dat één persoon zó kan innoveren?
КА: Насколько это здорово? Элон, как вы это сделали? Эти проекты такие — Paypal, SolarCity, Tesla, SpaceX, они настолько впечатляюще разные, они такие амбициозные проекты по масштабу. Как это возможно чтобы один человек смог внедрить инновации таким образом?
What is it about you?
Wat heb jij?

And the week before I showed up, the CEO of this big software company went to that group, 200 engineers, and canceled the project. And I stood there in front of 200 of the most depressed people I'v

In [9]:
# inseq seems to only work when doing 'pip install -r requirements.txt'
import inseq
from inseq.commands.attribute_context.attribute_context import attribute_context_with_model, AttributeContextArgs

Load inseq with mbart large en to nl

In [14]:
inseq_model = inseq.load_model(
    "facebook/mbart-large-50-one-to-many-mmt",
    "saliency",
    tokenizer_kwargs={'src_lang': 'en_XX', 'tgt_lang': 'ru_RU'},
)

In [15]:
def get_pecore_args(
        input_context_text,
        input_current_text,
        output_context_text,
        output_current_text=None,
        sample_identifier=None,
        output_folder='output'

):
    return AttributeContextArgs(
        model_name_or_path="facebook/mbart-large-50-one-to-many-mmt",
        attribution_method="saliency",
        attributed_fn="contrast_prob_diff",
        context_sensitivity_metric="kl_divergence",
        context_sensitivity_std_threshold=0,
        attribution_std_threshold=2,
        attribution_topk=5,
        input_context_text=input_context_text,
        input_current_text=input_current_text,
        output_context_text=output_context_text,
        # output_current_text=output_current_text,
        contextless_input_current_text="""{current}""",
        input_template="""{context} {current}""",
        contextless_output_current_text="""{current}""",
        output_template="{context} {current}",
        save_path=f"{output_folder}/{sample_identifier}.json",
        viz_path=f"{output_folder}/{sample_identifier}.html",
        tokenizer_kwargs={'src_lang': 'en_XX', 'tgt_lang': 'ru_RU'},
    )

Run little test

In [16]:
pec_args = get_pecore_args("This is context", "for translating this text", "Dit is context", sample_identifier="sample")

In [17]:
out = attribute_context_with_model(pec_args, inseq_model)

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  This is context
Input current: for translating this text
Output context: Dit is context
Output current: for translating this text

#1.
Generated output (CTI > 0.633):  for(2.971) translating this text
Input context (CCI > 0.094):     This is context
Output context (CCI > 0.100):    Dit is context
#2.
Generated output (CTI > 0.633):  for trans(0.790)lating this text
Input context (CCI > 0.058):     This is context
Output context (CCI > 0.060):    Dit is context

Now for use with the dataset

In [18]:
def get_pecore_args_for_row(row, sample_identifier=None, output_folder='output'):
    return get_pecore_args(
        row['context_concatenated']['en'],
        row['translation']['en'],
        row['context_concatenated']['ru'],
        sample_identifier=sample_identifier,
        output_folder=output_folder
    )

In [19]:
for i, row in enumerate(data):
    try:
        pecore_args = get_pecore_args_for_row(row, f"{i}_ru")
        out = attribute_context_with_model(pecore_args, inseq_model)
    except ValueError as e:
        print()
        print(f"ERROR for {i}")
        print(e)
        print()
        

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  CA: How cool is that?  Elon, how have you done this? These projects are so -- Paypal, SolarCity, 
Tesla, SpaceX, they're so spectacularly different, they're such ambitious projects at scale. How on Earth has one 
person been able to innovate in this way?
Input current: What is it about you?
Output context: КА: Насколько это здорово? Элон, как вы это сделали? Эти проекты такие — Paypal, SolarCity, Tesla, 
SpaceX, они настолько впечатляюще разные, они такие амбициозные проекты по масштабу. Как это возможно чтобы один 
человек смог внедрить инновации таким образом?
Output current: Что это о вас?

#1.
Generated output (CTI > 0.175):  Что(0.360) это о вас?
Input context (CCI > 0.005):     CA: How cool is that? Elon, how have you done this? These projects are so 
--(0.005) Paypal, SolarCity, Tesla, SpaceX, they're so spectacularly different, they're such ambitious projects at 
scale. How on Earth has one person been able to innovate in this way?(0.008)
Output context (CCI > 0.005):    КА: Насколько это здорово? Элон, как вы это сделали? Эти проекты такие — Paypal, 
SolarCity, Tesla, SpaceX, они настолько впечатляюще разные, они такие амбициозные проекты по масштабу. Как это 
возможно чтобы один человек смог внедрить инновации таким образом?(0.017)
#2.
Generated output (CTI > 0.175):  Что это о(0.261) вас?
Input context (CCI > 0.005):     CA: How cool is that? Elon, how have you done this? These projects are so 
--(0.010) Paypal, SolarCity, Tesla, SpaceX, they're so spectacularly different, they're such ambitious projects at 
scale. How on Earth has one person been able to innovate in this way(0.007)?(0.010)
Output context (CCI > 0.004):    КА: Насколько это здорово? Элон, как вы это сделали? Эти проекты такие — Paypal, 
SolarCity, Tesla, SpaceX, они настолько впечатляюще разные, они такие амбициозные проекты по масштабу. Как это 
возможно чтобы один человек смог вне(0.004)дрить инновации таким образом?(0.006)

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  And the week before I showed up, the CEO of this big software company went to that group, 200 
engineers, and canceled the project. And I stood there in front of 200 of the most depressed people I've ever 
talked to. And I described to them some of these Lego experiments, and they said they felt like they had just been 
through that experiment.
Input current: And I asked them, I said, "How many of you now show up to work later than you used to?"
Output context: И за неделю до того, как я появился, генеральный директор этой крупной программной компании пошел в
эту группу, 200 инженеров, и отменил проект. И я стоял там перед 200 наиболее депрессивными людьми, с которыми я 
когда-либо разговаривал. И я описал им некоторые из этих LEGO экспериментов, и они сказали, что они чувствовали, 
что они только что прошли этот эксперимент.
Output current: И я спросил их, я сказал, "Кто из вас теперь приезжает работать позже, чем раньше?"

#1.
Generated output (CTI > 0.176):  И(1.137) я спросил их, я сказал, "Кто из вас теперь приезжает работать позже, чем 
раньше?"
Input context (CCI > 0.006):     And the week before I showed up, the CEO of this big software company went to that
group, 200 engineers, and canceled the project. And I stood there in front of 200 of the most depressed people I've
ever talked to. And I described to them some of these Lego experiments, and they said they felt like they had just 
been through that experiment(0.009).(0.012)
Output context (CCI > 0.006):    И за неделю до того, как я появился, генеральный директор этой крупной программной
компании пошел в эту группу, 200 инженеров, и отменил проект. И я стоял там перед 200 наиболее депрессивными 
людьми, с которыми я когда-либо разговаривал. И(0.007) я описал им некоторые из этих LEGO экспериментов, и они 
сказали, что они чувствовали, что они только что прошли этот эксперимент.(0.021)
#2.
Generated output (CTI > 0.176):  И я спросил их, я(0.413) сказал, "Кто из вас теперь приезжает работать позже, чем 
раньше?"
Input context (CCI > 0.002):     And the week before I showed up, the CEO of this big software company went to that
group, 200 engineers, and canceled the project. And I stood there in front of 200 of the most depressed people I've
ever talked to. And I(0.003) described to them some of these(0.003) Lego experiments, and they said they felt like 
they had just been through that experiment.
Output context (CCI > 0.003):    И за неделю до того, как я появился, генеральный директор этой крупной программной
компании пошел в эту группу, 200 инженеров, и отменил проект. И я стоял там перед 200 наиболее депрессивными 
людьми, с которыми я когда-либо разговаривал. И я описа(0.003)л им некоторые из этих LEGO экспериментов, и они 
сказали, что они чувствовал(0.003)и, что они только что прошли(0.004) этот(0.003) эксперимент.(0.003)
#3.
Generated output (CTI > 0.176):  И я(0.398) спросил их, я сказал, "Кто из вас теперь приезжает работать позже, чем 
раньше?"
Input context (CCI > 0.001):     And the week before I showed up, the CEO of this big software company went to that
group, 200 engineers, and canceled the project. And I stood there in front of 200 of the most depressed people I've
ever talked to. And I described to them some of these Lego experiments, and they said they felt like they had just 
been through that experiment(0.002).(0.003)
Output context (CCI > 0.002):    И за неделю до того, как я появился, генеральный директор этой крупной программной
компании пошел в эту группу, 200 инженеров, и отменил проект. И я стоял там перед 200 наиболее депрессивными 
людьми, с которыми я когда-либо разговаривал. И я(0.002) описа(0.003)л им некоторые из этих LEGO экспериментов, и 
они сказали(0.002), что они чувствовал(0.002)и, что они только что прошли этот экспериме

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  [Woman: There is no Paradise! It only exists in your head!] [Man: God forbid!] [May God forgive 
you.]
Input current: [If you were not Abu Azzam's daughter ...]
Output context: Женщина: Рая не существует! он существует только в вашей голове! Мужчина: Бог запрещает! Да простит
тебя Бог.
Output current: Если бы ты не была дочерью Абу Аззама...

#1.
Generated output (CTI > 0.215):  Если(2.261) бы ты не была дочерью Абу Аззама...
Input context (CCI > 0.006):     [Woman: There is no Paradise! It only exists in your head!] [Man: God forbid!] 
[May God forgive you.](0.007)
Output context (CCI > 0.007):    Женщина: Рая не существует! он существует только в вашей голове! Мужчина: Бог 
запрещает! Да простит тебя Бог.(0.016)
#2.
Generated output (CTI > 0.215):  Если бы ты(0.390) не была дочерью Абу Аззама...
Input context (CCI > 0.011):     [Woman: There is no Paradise! It only exists in your head!] [Man: God forbid!] 
[May God forgive you.]
Output context (CCI > 0.013):    Женщина: Рая не существует! он существует только в вашей(0.025) голове! Мужчина: 
Бог запрещает! Да простит тебя(0.025) Бог.

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  My failure to procure myself a cup of sweet, green tea was not due to a simple misunderstanding. 
This was due to a fundamental difference in our ideas about choice. From my American perspective, when a paying 
customer makes a reasonable request based on her preferences, she has every right to have that request met.
Input current: The American way, to quote Burger King, is to "have it your way," because, as Starbucks says, 
"happiness is in your choices."
Output context: Моя неудача приобрести себе чашку сладкого зеленого чая не была причиной простого недоразумения. 
Это было связано с фундаментальным различием в наших мыслях о выборе. С моей американской точки зрения, когда 
оплачивающий клиент делает разумный запрос на основе своих предпочтений, у него есть все права на удовлетворение 
этого запроса.
Output current: Американский путь, чтобы процитировать Burger King, это "иметь его свой путь", потому что, как 
говорит Starbucks, "счастье в ваших выборах."

#1.
Generated output (CTI > 0.136):  Американски(0.875)й путь, чтобы процитировать Burger King, это "иметь его свой 
путь", потому что, как говорит Starbucks, "счастье в ваших выборах."
Input context (CCI > 0.007):     My failure to procure myself a cup of sweet, green tea was not due to a simple 
misunderstanding. This was due to a fundamental difference in our ideas about choice. From my American perspective,
when a paying customer makes a reasonable request based on her preferences, she has every right to have that 
request met(0.007).(0.008)
Output context (CCI > 0.008):    Моя неудача приобрести себе чашку сладкого зеленого чая не была причиной простого 
недоразумения. Это было связано с фундаментальным различием в наших мыслях о выборе. С моей американ(0.010)ской 
точки зрения, когда оплачивающий клиент делает разумный запрос на основе своих предпочтений, у него есть все права 
на удовлетворение этого запроса.(0.028)
#2.
Generated output (CTI > 0.136):  Американский путь, чтобы процитировать Burger King, это "иметь его(0.620) свой 
путь", потому что, как говорит Starbucks, "счастье в ваших выборах."
Input context (CCI > 0.003):     My failure to(0.004) procure myself a cup of sweet, green tea was not due to a 
simple misunderstanding(0.004). This was due to a fundamental difference in our ideas about choice(0.006). From my 
American perspective, when a paying(0.003) customer makes a(0.003) reasonable request based on her preferences, she
has every right to have that request met.
Output context (CCI > 0.002):    Моя неудача приобрести себе чашку сладкого зеленого чая не была причиной простого 
недоразумения. Это было связано с фундаментальным различием в наших мыслях о выборе. С моей американской точки 
зрения, когда оплачивающий клиент делает разумный запрос(0.003) на основе своих предпочтений, у него есть все права
на удовлетворение этого запроса.
#3.
Generated output (CTI > 0.136):  Американский путь, чтобы(0.549) процитировать Burger King, это "иметь его свой 
путь", потому что, как говорит Starbucks, "счастье в ваших выборах."
Input context (CCI > 0.004):     My failure to(0.005) procure myself a cup of sweet, green(0.005) tea was not due 
to a simple misunderstanding(0.005). This was due to a fundamental difference in our ideas about choice(0.006). 
From my American perspective, when a paying customer makes a reasonable request based on her preferences, she has 
every right to have that request met(0.005).
Output context (CCI > 0.004):    Моя неудач(0.005)а приобрести(0.005) себе чашку сладкого зеленого чая не была 
причиной простого недоразумения. Это было связано с фундаментальным различием в наших мыслях о выборе. С моей 
американской точки зрения, когда оплачивающий клиент делает разумный запрос на основе своих предпочтений, у него 
есть все права на 

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  They said they were going to beat us up if we didn't make some gunpowder for them. We said, well, 
what are you going to do with it? They said, we're going to melt it down and make rocket fuel.
Input current: Sure. We'll make you a big batch.
Output context: Они сказали, что изобьют нас, если мы не сделаем для них немного пороха. Мы сказали, хорошо, что вы
будете делать с этим? Они сказали, что мы собираемся расплавить его и сделать ракетное топливо.
Output current: Конечно. Мы сделаем вам большую партию.

#1.
Generated output (CTI > 0.167):  Конечно(1.263). Мы сделаем вам большую партию.
Input context (CCI > 0.009):     They said they were going to beat us up if we didn't make some gunpowder for them.
We said, well, what are you going to do with it? They said, we're going to melt it down and make rocket 
fuel.(0.024)
Output context (CCI > 0.009):    Они сказали, что изобьют нас, если мы не сделаем для них немного(0.010) пороха. Мы
сказали, хорошо, что вы будете делать с этим? Они сказали, что мы собираемся расплавить его и сделать ракетное 
топливо.(0.020)

...

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  And the rich man -- oh, dear -- crestfallen. "You mean God didn't want my bread?" And the poor man 
said, "And you mean God didn't answer my pleas?" And the rabbi said, "You've misunderstood me.
Input current: You've misunderstood totally," he said.
Output context: И богач - о боже! - пал духом. "Ты хочешь сказать, что Богу не нужен был мой хлеб?" И бедняк 
сказал: "И ты хочешь сказать, что Бог не ответил на мои мольбы?" И раввин сказал: "Вы меня неправильно поняли.
Output current: Вы совершенно неправильно поняли", - сказал он.

#1.
Generated output (CTI > 0.745):  Вы совершенно неправильно поняли",(3.095) - сказал он.
Input context (CCI > 0.003):     And the rich man -- oh, dear -- crestfallen. "You mean God didn't want my bread?" 
And the poor man said, "And you mean God didn't answer my pleas?" And the rabbi said, "You've misunderstood me.
Output context (CCI > 0.004):    И богач - о боже!(0.008) - пал духом. "Ты хочешь сказать, что Богу не нужен был 
мой хлеб?" И бедняк сказал: "И ты хочешь сказать, что Бог не ответил на мои мольбы?" И раввин сказал: "(0.005)Вы 
меня неправильно понял(0.004)и.(0.004)
#2.
Generated output (CTI > 0.745):  Вы(2.022) совершенно неправильно поняли", - сказал он.
Input context (CCI > 0.005):     And the rich man -- oh, dear -- crestfallen. "You mean God didn't want my bread?" 
And the poor man said, "And you mean God didn't answer my pleas?" And the rabbi said, "You've misunderstood me.
Output context (CCI > 0.006):    И богач - о боже! - пал духом. "Ты хочешь сказать, что Богу не нужен был мой 
хлеб?" И бедняк сказал: "И ты хочешь сказать, что Бог не ответил на мои мольбы?" И раввин сказал: "Вы(0.017) меня 
неправильно(0.011) поняли.(0.010)
#3.
Generated output (CTI > 0.745):  Вы совершенно неправильно(0.864) поняли", - сказал он.
Input context (CCI > 0.011):     And the rich man -- oh, dear -- crestfallen. "You mean God didn't want my bread?" 
And the poor man said, "And you mean God didn't answer my pleas?" And the rabbi said, "You've misunderstood me.
Output context (CCI > 0.012):    И богач - о боже! - пал духом. "Ты хочешь сказать, что Богу не нужен был мой 
хлеб?" И бедняк сказал: "И ты хочешь сказать, что Бог не ответил на мои мольбы?" И раввин сказал: "Вы меня 
неправильно(0.048) понял(0.012)и.

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  It wasn't this arm-wrestle, but it was a little surprising for the person involved. I interviewed 
Steve Martin. It wasn't all that long ago. And we were sitting there, and almost toward the beginning of the 
interview, I turned to him and I said, "Steve," or "Mr. Martin, it is said that all comedians have unhappy 
childhoods.
Input current: Was yours unhappy?"
Output context: Это был не настоящий армрестлинг, но для человека, участвовавшего в нем, это было немного 
неожиданно. Я брал интервью у Стива Мартина. Это было не так давно. Мы сидели там, и почти в самом начале интервью 
я повернулся к нему и сказал: "Стив", или "Мистер Мартин, говорят, что у всех комиков несчастливое детство.
Output current: Было ли ваше несчастливое?"

#1.
Generated output (CTI > 1.384):  Было ли ваше несчастливое?"(3.405)
Input context (CCI > 0.020):     It wasn't this arm-wrestle, but it was a little surprising for the person 
involved. I interviewed Steve Martin. It wasn't all that long ago. And we were sitting there, and almost toward the
beginning of the interview, I turned to him and I said, "Steve," or "Mr. Martin, it is said that all comedians have
unhappy childhoods.
Output context (CCI > 0.023):    Это был не настоящий армрестлинг, но для человека, участвовавшего в нем, это было 
немного неожиданно. Я брал интервью у Стива Мартина. Это было не так давно. Мы сидели там, и почти в самом начале 
интервью я повернулся к нему и сказал: "Стив", или "Мистер Мартин, говорят, что у всех комиков несчастливое 
детство(0.097).
#2.
Generated output (CTI > 1.384):  Было ли ваше нес(2.738)частливое?"
Input context (CCI > 0.002):     It wasn't this arm-wrestle, but it was a little surprising for the person 
involved. I interviewed Steve Martin. It wasn't all that long ago. And we were sitting there, and almost toward the
beginning of the interview, I turned to him and I said, "Steve," or "Mr. Martin, it is said that all comedians have
un(0.002)happy childhoods.(0.002)
Output context (CCI > 0.002):    Это был не настоящий армрестлинг, но для человека, участвовавшего в нем, это было 
немного неожиданно. Я брал интервью у Стива Мартина. Это было не так давно. Мы сидели там, и почти в самом начале 
интервью я повернулся к нему и сказал: "Стив", или "Мистер Мартин, говорят, что у всех комиков 
нес(0.006)част(0.002)ливое детство(0.004).
#3.
Generated output (CTI > 1.384):  Было ли ваше несчастливое(1.952)?"
Input context (CCI > 0.018):     It wasn't this arm-wrestle, but it was a little surprising for the person 
involved. I interviewed Steve Martin. It wasn't all that long ago. And we were sitting there, and almost toward the
beginning of the interview, I turned to him and I said, "Steve," or "Mr. Martin, it is said that all comedians have
unhappy childhoods.
Output context (CCI > 0.020):    Это был не настоящий армрестлинг, но для человека, участвовавшего в нем, это было 
немного неожиданно. Я брал интервью у Стива Мартина. Это было не так давно. Мы сидели там, и почти в самом начале 
интервью я повернулся к нему и сказал: "Стив", или "Мистер Мартин, говорят, что у всех комиков 
несчаст(0.021)ливо(0.035)е(0.032) детство(0.065).
#4.
Generated output (CTI > 1.384):  Было ли ваше несчастливо(1.443)е?"
Input context (CCI > 0.018):     It wasn't this arm-wrestle, but it was a little surprising for the person 
involved. I interviewed Steve Martin. It wasn't all that long ago. And we were sitting there, and almost toward the
beginning of the interview, I turned to him and I said, "Steve," or "Mr. Martin, it is said that all comedians have
unhappy childhoods.
Output context (CCI > 0.021):    Это был не настоящий армрестлинг, но для человека, участвовавшего в нем, это было 
немного неожиданно. Я брал интервью у Стива Мартина. Это было не так давно. Мы сидели там

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  Not based upon the quality of your work, how long you've been there, if you stink, just if you're 
gay, lesbian, bisexual or transgendered. All of which flies in the face of the gay agenda, also known as the U.S. 
Constitution. Specifically, this little amendment right here: "No state shall make or enforce any law which shall 
abridge the privileges or immunities of citizens of the United States." I'm looking at you, North Carolina.
Input current: But you're not looking at the U.S. Constitution.
Output context: Не зависит от качества вашей работы, от того, как долго вы там работаете, воняете ли вы, просто от 
того, являетесь ли вы геем, лесбиянкой, бисексуалом или трансгендером. Все это противоречит гей-программе, также 
известной как Конституция США. В частности, вот эта небольшая поправка: "Ни один штат не должен издавать или 
обеспечивать соблюдение законов, которые ограничивают привилегии или иммунитеты граждан Соединенных Штатов". Я 
смотрю на тебя, Северная Каролина.
Output current: Но вы не смотрите на Конституцию США.

#1.
Generated output (CTI > 0.136):  Но вы(0.656) не смотрите на Конституцию США.
Input context (CCI > 0.015):     Not based upon the quality of your work, how long you've been there, if you stink,
just if you're gay, lesbian, bisexual or transgendered. All of which flies in the face of the gay agenda, also 
known as the U.S. Constitution. Specifically, this little amendment right here: "No state shall make or enforce any
law which shall abridge the privileges or immunities of citizens of the United States." I'm looking at you, North 
Carolina.
Output context (CCI > 0.017):    Не зависит от качества вашей работы, от того, как долго вы там работаете, воняете 
ли вы, просто от того, являетесь ли вы геем, лесбиянкой, бисексуалом или трансгендером. Все это противоречит 
гей-программе, также известной как Конституция США. В частности, вот эта небольшая поправка: "Ни один штат не 
должен издавать или обеспечивать соблюдение законов, которые ограничивают привилегии или иммунитеты граждан 
Соединенных Штатов". Я смотрю на тебя(0.087), Северная Каролина.
#2.
Generated output (CTI > 0.136):  Но(0.318) вы не смотрите на Конституцию США.
Input context (CCI > 0.003):     Not based upon the quality of your work, how long you've been there, if you stink,
just if you're gay, lesbian, bisexual or transgendered. All of which flies in the face of the gay agenda, also 
known as the U.S. Constitution. Specifically, this little amendment right here: "No state shall make or enforce any
law which shall abridge the privileges or immunities of citizens of the United States." I'm looking at you, 
North(0.004) Carolina(0.007).(0.010)
Output context (CCI > 0.003):    Не зависит(0.004) от качества вашей работы, от того, как долго вы там работаете, 
воняете ли вы, просто от того, являетесь ли вы геем, лесбиянкой, бисексуалом или трансгендером. Все это 
противоречит гей-программе, также известной как Конституция США. В частности, вот эта небольшая поправка: "Ни один 
штат не должен издавать или обеспечив(0.004)ать соблюдение законов, которые ограничивают привилегии или иммунитеты 
граждан Соединенных Штатов". Я смотрю на тебя, Северная Каролина.(0.013)
#3.
Generated output (CTI > 0.136):  Но вы не смотрите на Конститу(0.136)цию США.
Input context (CCI > 0.006):     Not based upon the quality of your work, how long you've been there, if you stink,
just if you're gay, lesbian, bisexual or transgendered. All of which flies in the face of the gay agenda, also 
known as the U.S. Constitution. Specifically, this little amendment right here: "No state shall make or enforce any
law which shall abridge the privileges or immunities of citizens of the United States." I'm looking at you, North 
Carolina.
Output context (CCI > 0.007):    Не з

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  And you know the reason why, of course. This is because of technology -- yeah. Any computer 
designers in the room?
Input current: Yeah well, you guys are making my life miserable because track pads used to be round, a nice round 
shape.
Output context: И вы, конечно, знаете причину этого. Это из-за технологий - да. Есть здесь компьютерные дизайнеры?
Output current: Да, вы, ребята, делаете мою жизнь miserable, потому что track pads были круглыми, красивый круглый 
вид.

#1.
Generated output (CTI > 0.075):  Да(0.791), вы, ребята, делаете мою жизнь miserable, потому что track pads были 
круглыми, красивый круглый вид.
Input context (CCI > 0.012):     And you know the reason why, of course. This is because of technology -- yeah. Any
computer designers in the room?(0.022)
Output context (CCI > 0.012):    И вы, конечно, знаете причину этого. Это из-за технологий - да(0.020). Есть здесь 
компьютерные дизайнеры?(0.016)
#2.
Generated output (CTI > 0.075):  Да, вы, ребята, делаете мою жизнь miserable, потому что track pads были круглыми, 
красивый(0.188) круглый вид.
Input context (CCI > 0.019):     And you know the reason why, of course. This is because of technology -- yeah. Any
computer designers in the room?
Output context (CCI > 0.021):    И вы, конечно, знаете причину этого. Это из-за технологий - да. Есть(0.026) 
здесь(0.034) компьютерные дизайнеры?
#3.
Generated output (CTI > 0.075):  Да, вы(0.172), ребята, делаете мою жизнь miserable, потому что track pads были 
круглыми, красивый круглый вид.
Input context (CCI > 0.016):     And you know the reason why, of course. This is because of technology -- yeah. Any
computer designers in the room?(0.036)
Output context (CCI > 0.014):    И вы, конечно, знаете причину этого. Это из-за технологий - да. Есть здесь 
компьютерные дизайнеры?
#4.
Generated output (CTI > 0.075):  Да, вы, ребята, делает(0.115)е мою жизнь miserable, потому что track pads были 
круглыми, красивый круглый вид.
Input context (CCI > 0.007):     And you know the reason why, of course. This is because of technology -- yeah. Any
computer designers in the room?
Output context (CCI > 0.007):    И вы, конечно, знаете причину этого. Это из-за технологий - да. Есть(0.009) 
здесь(0.014) компьютерные дизайнеры?
#5.
Generated output (CTI > 0.075):  Да, вы,(0.092) ребята, делаете мою жизнь miserable, потому что track pads были 
круглыми, красивый круглый вид.
Input context (CCI > 0.006):     And you know the reason why, of course. This is because of technology -- yeah. Any
computer designers in the room?(0.007)
Output context (CCI > 0.006):    И вы,(0.007) конечно(0.008), знаете причину этого. Это из-за технологий - да. Есть
здесь компьютерные дизайнеры?
#6.
Generated output (CTI > 0.075):  Да,(0.087) вы, ребята, делаете мою жизнь miserable, потому что track pads были 
круглыми, красивый круглый вид.
Input context (CCI > 0.006):     And you know the reason why, of course. This is because of technology -- yeah. Any
computer designers in the room?(0.012)
Output context (CCI > 0.006):    И вы, конечно, знаете причину этого. Это из-за технологий - да. Есть здесь(0.006) 
компьютерные дизайнеры?
#7.
Generated output (CTI > 0.075):  Да, вы, ребята, делаете(0.083) мою жизнь miserable, потому что track pads были 
круглыми, красивый круглый вид.
Input context (CCI > 0.002):     And you know the reason why, of course. This is because of technology -- yeah. Any
computer designers in the room?
Output context (CCI > 0.002):    И вы, конечно, знаете причину этого. Это из-за технологий - да. Есть(0.003) 
здесь(0.002) компьютерные дизайнеры?
#8.
Generated output (CTI > 0.075):  Да, вы, ребята, делаете мою жизнь miserable, потому что track(0.078) pads были 
круглыми, красивый круглый вид.
Input context (CCI > 0.005):     And you know the re

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  Hi, sir, could you stand up, please? Just right where you are. You're married, you follow 
directions well. Nice to meet you, sir.
Input current: You don't have a lot in your pockets. Anything down here?
Output context: Здравствуйте, сэр, не могли бы вы встать, пожалуйста? Оставайтесь на месте. Вы женаты, вы хорошо 
следуете инструкциям. Приятно познакомиться, сэр.
Output current: У вас не много в кармане.

#1.
Generated output (CTI > 0.483):  У(2.316) вас не много в кармане.
Input context (CCI > 0.011):     Hi, sir, could you stand up, please? Just right where you are. You're married, you
follow directions well. Nice to meet you, sir.
Output context (CCI > 0.013):    Здравствуйте(0.019), сэр, не могли бы вы встать, пожалуйста? Оставайтесь на месте.
Вы женаты, вы хорошо следуете инструкциям. Приятно познакомиться, сэр(0.015).(0.028)
#2.
Generated output (CTI > 0.483):  У вас(0.582) не много в кармане.
Input context (CCI > 0.004):     Hi, sir, could you stand up, please? Just right where you are. You're married, you
follow directions well. Nice to meet you, sir.
Output context (CCI > 0.004):    Здравствуйте(0.010), сэр, не могли бы вы встать, пожалуйста? Оставайтесь(0.005) на
месте. Вы(0.005) женаты, вы хорошо следуете инструкциям. Приятно познакомиться, сэр.